In [1]:
!pip install segmentation_models_pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 9.1 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=87dda17cab7bd2ad62d1c26fd17a44e311814d965da2b82b9509c701a15dcdec
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=91670d05fe8b9cbc15b01d85cede8fdd0d62107be6cb1a9ddca21379090d0ce8
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels


In [2]:
import pandas as pd
import os 
import cv2
import numpy as np
import torch
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = smp.Unet(encoder_name="resnet34", encoder_weights=None, in_channels=1, classes=5)
model.load_state_dict(torch.load("/kaggle/input/heart-segmwntation/pytorch/default/1/unet_heart_segmentation.pth"))
model.to(device)
model.eval() 

<ipython-input-4-0d348a2eee1c>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/heart-segmwntation/pytorch/default/1/unet_hea

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [5]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=1), 
    transforms.Resize((256, 256)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  
])


In [6]:
def preprocess_frame(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    input_tensor = transform(gray_frame).unsqueeze(0).to(device) 
    return input_tensor

In [7]:
def get_segmentation_mask(frame):
    input_tensor = preprocess_frame(frame)
    with torch.no_grad():
        output = model(input_tensor) 
        pred_mask = torch.argmax(output, dim=1).cpu().numpy()[0]  
    h, w = frame.shape[:2]
    pred_mask = cv2.resize(pred_mask.astype(np.uint8), (w, h), interpolation=cv2.INTER_NEAREST)
    return pred_mask


In [8]:
def get_contours(mask, chamber_id):
    chamber_mask = (mask == chamber_id).astype(np.uint8) * 255
    contours, _ = cv2.findContours(chamber_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    filtered_contours = [c for c in contours if cv2.contourArea(c) > 200]  
    
    return filtered_contours

In [9]:
def calculate_polygon_area(contours):
    total_area = 0
    for contour in contours:
        if len(contour) >= 3:
            total_area += cv2.contourArea(contour)
    return total_area

In [10]:
def smooth_scale(value, ref):
    if ref == 0:
        return 0
    scale_factor = value / ref

    if scale_factor < 0.70:
        scale_factor = 0.70 + (scale_factor - 0.70) * 0.001  
    elif scale_factor > 1.30:
        scale_factor = 1.30 - (scale_factor - 1.30) * 0.001

    return round(scale_factor, 4)

In [11]:
def extract_smoothed_end_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    raw_areas = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        mask = get_segmentation_mask(frame)
        contours_dict = {ch: get_contours(mask, ch) for ch in range(1, 5)}
        areas_for_frame = {ch: calculate_polygon_area(contours_dict[ch]) for ch in range(1, 5)}
        total_area = sum(areas_for_frame.values())

        raw_areas.append({"Frame": frame_idx, "Total Area": total_area, **areas_for_frame})
        frame_idx += 1

    cap.release()

    if len(raw_areas) < 2:
        raise ValueError(f"Video {video_path} does not have enough frames to compute diastole and systole.")

    total_areas = [frame["Total Area"] for frame in raw_areas]

    end_diastole_idx = total_areas.index(max(total_areas))  
    end_systole_idx = total_areas.index(min(total_areas[end_diastole_idx:]))

    if end_systole_idx <= end_diastole_idx:
        end_systole_idx = end_diastole_idx + 1
        if end_systole_idx >= len(total_areas):
            end_systole_idx = len(total_areas) - 1

    stable_frames = raw_areas[end_diastole_idx:end_systole_idx]
    if not stable_frames:
        reference_frame = raw_areas[end_diastole_idx]
    else:
        reference_frame = min(stable_frames, key=lambda f: abs(f["Total Area"] - np.mean(total_areas)))

    scaled_dia = {}
    scaled_sys = {}

    for chamber_id in range(1, 5):
        dia_value = raw_areas[end_diastole_idx][chamber_id]
        sys_value = raw_areas[end_systole_idx][chamber_id]
        ref_value = reference_frame[chamber_id] or 1 

        scaled_dia[chamber_id] = smooth_scale(dia_value, ref_value)
        scaled_sys[chamber_id] = smooth_scale(sys_value, ref_value)

    return scaled_dia, scaled_sys


In [12]:
def process_multiple_videos_smoothed(video_list):
    all_dia = {1: [], 2: [], 3: [], 4: []}
    all_sys = {1: [], 2: [], 3: [], 4: []}

    for video_path in video_list:
        scaled_dia, scaled_sys = extract_smoothed_end_frames(video_path)
        for ch in range(1, 5):
            all_dia[ch].append(scaled_dia[ch])
            all_sys[ch].append(scaled_sys[ch])

    avg_scaled_dia = {ch: round(np.mean(all_dia[ch]), 4) for ch in range(1, 5)}
    avg_scaled_sys = {ch: round(np.mean(all_sys[ch]), 4) for ch in range(1, 5)}

    chamber_names = {
        1: "Left Ventricle (LV)",
        2: "Left Atrium (LA)",
        3: "Right Ventricle (RV)",
        4: "Right Atrium (RA)"
    }

    print("\n--- AVERAGE SMOOTHED SCALED END DIASTOLIC ---")
    for ch in range(1, 5):
        print(f"{chamber_names[ch]}: {avg_scaled_dia[ch]}")

    print("\n--- AVERAGE SMOOTHED SCALED END SYSTOLIC ---")
    for ch in range(1, 5):
        print(f"{chamber_names[ch]}: {avg_scaled_sys[ch]}")

In [13]:
cluster_df = pd.read_csv("/kaggle/input/video-cluster/output_clusters.csv")

In [14]:
cluster_0_videos = cluster_df[cluster_df['cluster'] == 0]['FileName'].tolist()

In [15]:
base_path = r"/kaggle/input/echonet-dynamic/EchoNet-Dynamic/Videos"

In [16]:
video_list = [os.path.join(base_path, f"{file}.avi") for file in cluster_0_videos]

In [17]:
len(video_list)

585

In [18]:
process_multiple_videos_smoothed(video_list)


--- AVERAGE SMOOTHED SCALED END DIASTOLIC ---
Left Ventricle (LV): 1.123
Left Atrium (LA): 1.027
Right Ventricle (RV): 1.0706
Right Atrium (RA): 0.9946

--- AVERAGE SMOOTHED SCALED END SYSTOLIC ---
Left Ventricle (LV): 0.8749
Left Atrium (LA): 0.9007
Right Ventricle (RV): 0.797
Right Atrium (RA): 0.8034
